## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df = city_data_df.drop(columns='Unnamed: 0')
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Upernavik,GL,72.7868,-56.1549,6.58,80,54,3.33,broken clouds
1,Muros,ES,42.7762,-9.0603,55.99,88,100,20.00,overcast clouds
2,Bluff,NZ,-46.6000,168.3333,57.00,67,0,1.99,clear sky
3,Cabedelo,BR,-6.9811,-34.8339,78.80,89,0,5.75,clear sky
4,Touros,BR,-5.1989,-35.4608,78.80,83,20,4.61,few clouds
...,...,...,...,...,...,...,...,...,...
691,Liverpool,GB,53.4106,-2.9779,51.80,76,75,25.32,broken clouds
692,Mayskiy,RU,47.6931,40.1025,17.60,79,0,6.71,clear sky
693,Makakilo City,US,21.3469,-158.0858,80.60,69,1,13.80,clear sky
694,Acuitzio,MX,20.3450,-101.9771,72.48,19,1,3.27,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,Cabedelo,BR,-6.9811,-34.8339,78.80,89,0,5.75,clear sky
4,Touros,BR,-5.1989,-35.4608,78.80,83,20,4.61,few clouds
5,Saint-Philippe,RE,-21.3585,55.7679,78.80,73,20,12.66,light rain
6,Davila,PH,18.4726,120.5763,82.40,57,75,4.61,broken clouds
10,Port-Gentil,GA,-0.7193,8.7815,81.18,83,89,10.89,overcast clouds
12,Vaini,TO,-21.2000,-175.2000,84.20,65,20,8.05,few clouds
22,Arraial Do Cabo,BR,-22.9661,-42.0278,77.00,88,75,2.30,broken clouds
29,Butaritari,KI,3.0707,172.7902,81.30,77,13,10.47,light rain
30,Kloulklubed,PW,7.0419,134.2556,80.60,89,75,3.44,broken clouds
32,Ambon,ID,-3.6954,128.1814,84.20,74,20,3.44,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   198
Country                198
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cabedelo,BR,78.80,clear sky,-6.9811,-34.8339,
4,Touros,BR,78.80,few clouds,-5.1989,-35.4608,
5,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
6,Davila,PH,82.40,broken clouds,18.4726,120.5763,
10,Port-Gentil,GA,81.18,overcast clouds,-0.7193,8.7815,
12,Vaini,TO,84.20,few clouds,-21.2000,-175.2000,
22,Arraial Do Cabo,BR,77.00,broken clouds,-22.9661,-42.0278,
29,Butaritari,KI,81.30,light rain,3.0707,172.7902,
30,Kloulklubed,PW,80.60,broken clouds,7.0419,134.2556,
32,Ambon,ID,84.20,few clouds,-3.6954,128.1814,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,
        "type": "lodging",
        "key": {g_key},}

In [8]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df= hotel_df.replace("", np.nan)
hotel_df= hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cabedelo,BR,78.80,clear sky,-6.9811,-34.8339,Hotel Pigalle
4,Touros,BR,78.80,few clouds,-5.1989,-35.4608,INN NEW HORIZON
5,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Davila,PH,82.40,broken clouds,18.4726,120.5763,Blue Ridge Beach Resort
10,Port-Gentil,GA,81.18,overcast clouds,-0.7193,8.7815,Fengshui Residence
...,...,...,...,...,...,...,...
673,Kapit,MY,77.11,overcast clouds,2.0167,112.9333,Hotel Meligai Kapit
676,Ciudad Bolivar,VE,83.46,overcast clouds,8.1222,-63.5497,Hotel Edi
677,Winneba,GH,82.40,few clouds,5.3511,-0.6231,Windy Lodge & Beach Resort
682,Bowen,AU,84.20,broken clouds,-20.0167,148.2333,Castle Motor Lodge Motel


In [10]:
# 8a. Create the output File (CSV)
hotel_df
# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
   
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[['Lat', 'Lng']]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))